In [1]:
import pandas as pd
import os

In [2]:
# Global variables
DATA_FOLDER = '../data/yellow_taxi'
data_files = [f for f in os.listdir(DATA_FOLDER) if os.path.isfile(os.path.join(DATA_FOLDER, f))]
print(data_files)

['yellow_tripdata_2022-01.parquet', 'yellow_tripdata_2022-02.parquet']


In [3]:
df_jan = pd.read_parquet(os.path.join(DATA_FOLDER, data_files[0]))
df_feb = pd.read_parquet(os.path.join(DATA_FOLDER, data_files[1]))

In [4]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [5]:
print(df_jan.dtypes)

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object


In [6]:
# Number of columns
print(len(df_jan.columns))

19


In [7]:
# Duration standard deviation
df_jan["duration"] = (df_jan["tpep_dropoff_datetime"] - df_jan["tpep_pickup_datetime"]).dt.seconds / 60
df_feb["duration"] = (df_feb["tpep_dropoff_datetime"] - df_feb["tpep_pickup_datetime"]).dt.seconds / 60

df_jan["duration"].std()

56.18570689262747

In [8]:
# Dataset fraction after filtering
full_dataset_count = len(df_jan.index)
df_jan_filtered = df_jan[(df_jan["duration"] > 1) & (df_jan["duration"] < 60)]
df_feb_filtered = df_feb[(df_feb["duration"] > 1) & (df_feb["duration"] < 60)]

filtered_dataset_count = len(df_jan_filtered.index)
print(full_dataset_count, filtered_dataset_count, (filtered_dataset_count / full_dataset_count) * 100)

2463931 2421031 98.25887981441038


In [30]:
# Export dataset as dictionary
df_jan_dataset = df_jan_filtered[['PULocationID', 'DOLocationID', 'duration']]
duration = df_jan_dataset.pop('duration')
dict_df_jan_dataset = df_jan_dataset.to_dict(orient='records')

In [31]:
print(dict_df_jan_dataset[:10])

[{'PULocationID': 142, 'DOLocationID': 236}, {'PULocationID': 236, 'DOLocationID': 42}, {'PULocationID': 166, 'DOLocationID': 166}, {'PULocationID': 114, 'DOLocationID': 68}, {'PULocationID': 68, 'DOLocationID': 163}, {'PULocationID': 138, 'DOLocationID': 161}, {'PULocationID': 233, 'DOLocationID': 87}, {'PULocationID': 238, 'DOLocationID': 152}, {'PULocationID': 166, 'DOLocationID': 236}, {'PULocationID': 236, 'DOLocationID': 141}]


In [32]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [33]:
dict_vect = DictVectorizer(sparse=False)
one_hot_enc = OneHotEncoder(handle_unknown='ignore')
X = dict_vect.fit_transform(dict_df_jan_dataset)
X_ohe = one_hot_enc.fit_transform(X)
y = duration.to_numpy()

In [34]:
print(np.shape(X_ohe))
print(np.shape(y))

(2421031, 515)
(2421031,)


In [35]:
#print(X_ohe)

In [36]:
# Train linear regression model
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_ohe, y)
model.score(X_ohe, y)

0.39714980887624485

In [38]:
from sklearn.metrics import mean_squared_error as mse
y_pred = model.predict(X_ohe)
print("RMSE training: ", np.sqrt(mse(y_pred, y)))

RMSE training:  6.983871708402315


In [39]:
# Validate the model
df_feb_dataset = df_feb_filtered[['PULocationID', 'DOLocationID', 'duration']]
duration = df_feb_dataset.pop('duration')
dict_df_feb_dataset = df_feb_dataset.to_dict(orient='records')
X_val = dict_vect.transform(dict_df_feb_dataset)
X_val_ohe = one_hot_enc.transform(X_val)
y_val = duration.to_numpy()
y_val_pred = model.predict(X_val_ohe)
print("RMSE validation: ", np.sqrt(mse(y_val_pred, y_val)))

RMSE validation:  7.784441370070872
